In [5]:
import json

iu_path = "/data/aofei/hallucination/IU_Xray/iu_xray/annotation.json"
iu_data = json.load(open(iu_path, "r"))
iu_data.keys()

dict_keys(['train', 'val', 'test'])

In [6]:
iu_data['train'][0], len(iu_data['train'])

({'id': 'CXR2384_IM-0942',
  'report': 'The heart size and pulmonary vascularity appear within normal limits. A large hiatal hernia is noted. The lungs are free of focal airspace disease. No pneumothorax or pleural effusion is seen. Degenerative changes are present in the spine.',
  'image_path': ['CXR2384_IM-0942/0.png', 'CXR2384_IM-0942/1.png'],
  'split': 'train'},
 2069)

In [7]:
s = 0
for i in range(len(iu_data['train'])):
    if len(iu_data['train'][i]['image_path']) == 1:
        s += 1
s

0

In [8]:
iu_data['test'][0], len(iu_data['test'])

({'id': 'CXR3030_IM-1405',
  'report': 'Normal cardiomediastinal silhouette. There is no focal consolidation. There are no XXXX of a large pleural effusion. There is no pneumothorax. There is no acute bony abnormality seen.',
  'image_path': ['CXR3030_IM-1405/0.png', 'CXR3030_IM-1405/1.png'],
  'split': 'test'},
 590)

In [9]:
# qs = qs.replace(DEFAULT_IMAGE_TOKEN, '').strip()
report_prompt_iu = "Analyze the chest X-ray and generate a concise medical report."
#training2
iu_train_data = iu_data['train']
new_iu_train_data = []
s = 0
for i in iu_train_data:
    template = dict()
    template['image'] = i['image_path'][0]
    template['id'] = str(i['id']) + "_train" + str(s)
    template['conversations'] = []
    new_qa = {"from": "human", "value": "<image>\n" + report_prompt_iu}
    new_qa2 = {"from": "gpt", "value": i['report']}
    template['conversations'] += [new_qa, new_qa2]
    new_iu_train_data.append(template)
    s+=1

new_iu_train_data[-1], len(new_iu_train_data)

({'image': 'CXR3835_IM-1938/0.png',
  'id': 'CXR3835_IM-1938_train2068',
  'conversations': [{'from': 'human',
    'value': '<image>\nAnalyze the chest X-ray and generate a concise medical report.'},
   {'from': 'gpt',
    'value': 'Lungs are clear bilaterally. There is no focal consolidation, pleural effusion, or pneumothoraces. There is widening adjacent to the right paratracheal stripe, most XXXX represents the SVC with rotated position. XXXX are unremarkable.'}]},
 2069)

In [10]:
iu_test_data = iu_data['test']
new_iu_test_data = []
s = 0
for i in iu_test_data:
    template = dict()
    template['image'] = i['image_path'][0]
    template['id'] = str(i['id']) + "_train" + str(s)
    template['conversations'] = []
    new_qa = {"from": "human", "value": "<image>\n" + report_prompt_iu}
    new_qa2 = {"from": "gpt", "value": i['report']}
    template['conversations'] += [new_qa, new_qa2]
    new_iu_test_data.append(template)
    s+=1

new_iu_test_data[-1], len(new_iu_test_data)

({'image': 'CXR49_IM-2110/0.png',
  'id': 'CXR49_IM-2110_train589',
  'conversations': [{'from': 'human',
    'value': '<image>\nAnalyze the chest X-ray and generate a concise medical report.'},
   {'from': 'gpt',
    'value': 'The heart, pulmonary XXXX and mediastinum are within normal limits. There is no pleural effusion or pneumothorax. There is no focal air space opacity to suggest a pneumonia. There are degenerative changes of the thoracic spine. There is a calcified granuloma identified in the right suprahilar region. The aorta is mildly tortuous and ectatic. There is asymmetric right apical smooth pleural thickening. There are severe degenerative changes of the XXXX.'}]},
 590)

In [11]:
import json
import csv
train_path = '/data/aofei/hallucination/IU_Xray/data_report/training.json'
test_path = '/data/aofei/hallucination/IU_Xray/data_report/test.json'
# Save training data
# print(len(new_iu_train_data))
# with open(train_path, 'w') as train_file:
#     json.dump(new_iu_train_data, train_file, indent=4)

# # Save test data
# print(len(new_iu_test_data))
# with open(test_path, 'w') as test_file:
#     json.dump(new_iu_test_data, test_file, indent=4)

csv_test_path = '/data/aofei/hallucination/IU_Xray/data_report/test.csv'

    # Write test data to CSV
with open(csv_test_path, 'w', newline='') as csvfile:
    fieldnames = ['study_id', 'report']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for data in new_iu_test_data:
        new_line = dict()
        new_line['study_id'] = int(data['id'].split("train")[-1])
        new_line['report'] = data['conversations'][1]['value']
        writer.writerow(new_line)

In [10]:
pred_path = "/data/aofei/hallucination/mitigation/IU_Xray/llava_med/lora_report/epoch4/seed4/inference/pred.jsonl"
with open(pred_path, "r") as f:
    pred_data = [json.loads(line) for line in f]
pred_data[0]

{'question_id': 'CXR3030_IM-1405_train0',
 'prompt': '<image>\nAnalyze the chest X-ray and generate a concise medical report.',
 'text': 'Assistant: Heart size is normal. The lungs are clear. There is no pleural line to suggest pneumothorax or costophrenic XXXX blunting to suggest large pleural effusion. Bony structures are within normal limits.',
 'gt': 'Normal cardiomediastinal silhouette. There is no focal consolidation. There are no XXXX of a large pleural effusion. There is no pneumothorax. There is no acute bony abnormality seen.',
 'answer_id': 'CHoTw6eoyGHDKy3eDricq5',
 'model_id': '/data/aofei/LLM/llava_med',
 'metadata': {}}

In [13]:

csv_pred_path = '/data/aofei/hallucination/mitigation/IU_Xray/llava_med/lora_report/epoch4/seed4/inference/pred.csv'

    # Write test data to CSV
with open(csv_pred_path, 'w', newline='') as csvfile:
    fieldnames = ['study_id', 'report']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for data in pred_data:
        new_line = dict()
        new_line['study_id'] = int(data['question_id'].split("train")[-1])
        new_line['report'] = data['text'].strip("Assistant:")
        writer.writerow(new_line)

In [3]:
pred_path = "/data/aofei/hallucination/mitigation/IU_Xray_report/llava_med/ours_v1/top128_balance_top3/all_expert_8_16_rank16/lora_0_bbox_0.06/epoch4_seed4/inference/pred_beam.jsonl"
with open(pred_path, "r") as f:
    pred_data = [json.loads(line) for line in f]
pred_data[0]

{'question_id': 'CXR3030_IM-1405_train0',
 'prompt': '<image>\nAnalyze the chest X-ray and generate a concise medical report.',
 'text': 'Assistant: The heart is normal in size. The mediastinum is unremarkable. The lungs are clear.',
 'gt': 'Normal cardiomediastinal silhouette. There is no focal consolidation. There are no XXXX of a large pleural effusion. There is no pneumothorax. There is no acute bony abnormality seen.',
 'answer_id': 'HJC3RdZNazP4gzgdvtxPdE',
 'model_id': '/data/aofei/LLM/llava_med',
 'metadata': {}}

In [4]:
csv_pred_path = '/data/aofei/hallucination/mitigation/IU_Xray_report/llava_med/ours_v1/top128_balance_top3/all_expert_8_16_rank16/lora_0_bbox_0.06/epoch4_seed4/inference/pred_beam.csv'

    # Write test data to CSV
with open(csv_pred_path, 'w', newline='') as csvfile:
    fieldnames = ['study_id', 'report']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for data in pred_data:
        new_line = dict()
        new_line['study_id'] = int(data['question_id'].split("train")[-1])
        new_line['report'] = data['text'].strip("Assistant:")
        writer.writerow(new_line)

In [1]:
import json

test_infer_path = "/data/aofei/hallucination/IU_Xray/data_report/test.json"
test_infer_data = json.load(open(test_infer_path, "r")) 

In [2]:
test_infer_data[0]

{'image': 'CXR3030_IM-1405/0.png',
 'id': 'CXR3030_IM-1405_train0',
 'conversations': [{'from': 'human',
   'value': '<image>\nAnalyze the chest X-ray and generate a concise medical report.'},
  {'from': 'gpt',
   'value': 'Normal cardiomediastinal silhouette. There is no focal consolidation. There are no XXXX of a large pleural effusion. There is no pneumothorax. There is no acute bony abnormality seen.'}]}

In [3]:
report_prompt_iu = "Analyze the chest X-ray and generate a medical report."
new_test_infer_data = []
for i in test_infer_data:
    i['conversations'][0]['value'] = "<image>\n" + report_prompt_iu
    new_test_infer_data.append(i)

In [4]:
new_test_infer_path = "/data/aofei/hallucination/IU_Xray/data_report/original_inference/test.json"
with open(new_test_infer_path, 'w') as test_file:
    json.dump(new_test_infer_data, test_file, indent=4)